In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
# fonction
def test_primary_key(df, column):
        if len(df) == len(df[column].drop_duplicates()):
                print("la clé  est unique")
        else : print('la clé n est pas unique')

# Nettoyage :Animaux et Végétaux

In [3]:
animaux=pd.read_csv("fr_animaux.csv")
vegetaux=pd.read_csv("fr_vegetaux.csv")

animaux['origine'] = 'animal'
vegetaux['origine'] = 'vegetal'
df = pd.concat([animaux, vegetaux])

test_primary_key(df, ['Zone', "Code zone", "Produit", "origine", "Élément"])

la clé  est unique


In [4]:
#df.head(10)
df['Élément'].value_counts()

Disponibilité intérieure                                         15478
Importations - Quantité                                          14946
Disponibilité alimentaire (Kcal/personne/jour)                   14333
Nourriture                                                       14107
Disponibilité alimentaire en quantité (kg/personne/an)           14107
Exportations - Quantité                                          12320
Disponibilité de matière grasse en quantité (g/personne/jour)    11877
Disponibilité de protéines en quantité (g/personne/jour)         11641
Production                                                        9271
Variation de stock                                                6857
Autres utilisations (non alimentaire)                             5571
Pertes                                                            4329
Aliments pour animaux                                             2757
Traitement                                                        2328
Semenc

### Liste de valeurs que prend "element" >> Nécessité de pivoter la table

In [5]:
df = df.pivot_table(index=['Zone', "Code zone", "Code Produit", "Produit", "origine"], values="Valeur", columns='Élément' )
df = df.rename_axis(None, axis=1)
df.reset_index(inplace = True)
# à renplacer par les bon nom
colonnes = ['Aliments pour animaux',\
            'Autres utilisations (non alimentaire)',\
            'Disponibilité intérieure',
            'Exportations - Quantité',\
            'Importations - Quantité',\
            'Nourriture','Pertes',\
            'Production',
            'Semences',\
            'Traitement',\
            'Variation de stock']

for i in colonnes: 
    df[i] *= 1000000
# on va transformer toutes ces valeurs en des colonnes individuelles

df.sample(5)

,Zone,Code zone,Code Produit,Produit,origine,Aliments pour animaux,Autres utilisations (non alimentaire),Disponibilité alimentaire (Kcal/personne/jour),Disponibilité alimentaire en quantité (kg/personne/an),Disponibilité de matière grasse en quantité (g/personne/jour),Disponibilité de protéines en quantité (g/personne/jour),Disponibilité intérieure,Exportations - Quantité,Importations - Quantité,Nourriture,Pertes,Production,Semences,Traitement,Variation de stock
7173,Japon,110,2617,Pommes,vegetal,NaN,NaN,11.0,16.17,0.05,0.09,2.187000e+09,23000000.0,1.468000e+09,2.056000e+09,74000000.0,742000000.0,NaN,56000000.0,NaN
88,Afrique du Sud,202,2562,Palmistes,vegetal,NaN,0.0,NaN,NaN,NaN,NaN,0.000000e+00,0.0,0.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN
15019,Zambie,251,2614,"Agrumes, Autres",vegetal,NaN,NaN,0.0,0.12,NaN,0.00,2.000000e+06,0.0,2.000000e+06,2.000000e+06,NaN,NaN,NaN,NaN,0.0
9826,Niger,158,2578,Huile de Coco,vegetal,NaN,NaN,0.0,0.00,0.00,NaN,0.000000e+00,NaN,0.000000e+00,0.000000e+00,NaN,NaN,NaN,NaN,NaN
2289,Burkina Faso,233,2558,Graines Colza/Moutarde,vegetal,NaN,NaN,0.0,0.00,0.00,0.00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,NaN,NaN,NaN,NaN,NaN


# Nettoyage sous alimentation

In [6]:
sousalimentation=pd.read_csv("fr_sousalimentation.csv")
sousalimentation = sousalimentation[['Zone', 'Code zone', 'Année', 'Valeur']].copy()
test_primary_key(sousalimentation, ['Zone', "Code zone", "Année"])
# la clé est bien unique

sousalimentation['Valeur'] = pd.to_numeric(sousalimentation['Valeur'], errors='coerce')
# quand il va tomber sur <0.1, qu'il ne peut pas convertir, il va les remplacer par des NA
sousalimentation['Valeur'] *= 1000000 

sousalimentation.rename(columns={'Valeur':'pop_sousalim'}, inplace=True)
sousalimentation['Année'] = pd.to_numeric(sousalimentation['Année'].str[0:4])+1
# plus facile pour filter sur 2017
sousalimentation.head(2)

la clé  est unique


,Zone,Code zone,Année,pop_sousalim
0,Afghanistan,2,2013,7900000.0
1,Afghanistan,2,2014,8800000.0


# Nettoyage de Population

In [7]:
population = pd.read_csv("fr_population.csv")
population = population[['Zone', 'Code zone', 'Année', 'Valeur']].copy()
test_primary_key(population, ['Zone', "Code zone", "Année"])
# la clé est bien unique
population.rename(columns={'Valeur':'population'}, inplace=True)
population['population'] *= 1000 
population.head(1)

la clé  est unique


,Zone,Code zone,Année,population
0,Afghanistan,2,2013,30552000


# Question 1 :

Population totales du monde

In [8]:
population['population'].sum()

8413993000

##### La population trop élevée.

In [9]:
population.sort_values(by='population', ascending=False)

,Zone,Code zone,Année,population
33,Chine,351,2013,1416667000
36,"Chine, continentale",41,2013,1385567000
75,Inde,100,2013,1252140000
55,États-Unis d'Amérique,231,2013,320051000
76,Indonésie,101,2013,249866000
...,...,...,...,...
89,Kiribati,83,2013,102000
6,Antigua-et-Barbuda,8,2013,90000
47,Dominique,55,2013,72000
20,Bermudes,17,2013,65000


In [10]:
population[population['Zone'].str.contains('Chine')]

,Zone,Code zone,Année,population
33,Chine,351,2013,1416667000
34,Chine - RAS de Hong-Kong,96,2013,7204000
35,Chine - RAS de Macao,128,2013,566000
36,"Chine, continentale",41,2013,1385567000
37,"Chine, Taiwan Province de",214,2013,23330000


##### Le soucis c'est que la Chinne là plusieurs fois. On va donc supprimer les redondances de la Chine.

In [11]:
# suppresions de 96 (RAS de Hong-Kong), 128 (RAS de Macao), etc
df = df.loc[~df['Code zone'].isin([96, 128, 41, 214]), :]
population = population.loc[~population['Code zone'].isin([96, 128, 41, 214]), :]
sousalimentation = sousalimentation.loc[~sousalimentation['Code zone'].isin([96, 128, 41, 214]), :]

totalPopulation = population['population'].sum()
print(int(totalPopulation),'de personnes sur Terre en 2013')

6997326000 de personnes sur Terre en 2013


# Question 2 

In [12]:
df_temp = df.loc[(df['Zone'] == 'France') & (df['Produit']== 'Blé'),:].iloc[0]
df_temp.head(100)

Zone                                                                    France
Code zone                                                                   68
Code Produit                                                              2511
Produit                                                                    Blé
origine                                                                vegetal
Aliments pour animaux                                             7822000000.0
Autres utilisations (non alimentaire)                             2824000000.0
Disponibilité alimentaire (Kcal/personne/jour)                           811.0
Disponibilité alimentaire en quantité (kg/personne/an)                  108.43
Disponibilité de matière grasse en quantité (g/personne/jour)              4.2
Disponibilité de protéines en quantité (g/personne/jour)                 25.99
Disponibilité intérieure                                         20298000000.0
Exportations - Quantité                             

In [13]:
print(df_temp["Production"] + df_temp["Importations - Quantité"] - df_temp["Exportations - Quantité"] + df_temp["Variation de stock"])

print(df_temp["Disponibilité intérieure"])

print(df_temp["Semences"] + df_temp["Pertes"] + df_temp["Nourriture"] + df_temp["Aliments pour animaux"] + df_temp["Traitement"] + df_temp["Autres utilisations (non alimentaire)"])

20298000000.0
20298000000.0
20298000000.0


In [14]:
del(df_temp)

# Question 3

In [15]:
df.head(1)

,Zone,Code zone,Code Produit,Produit,origine,Aliments pour animaux,Autres utilisations (non alimentaire),Disponibilité alimentaire (Kcal/personne/jour),Disponibilité alimentaire en quantité (kg/personne/an),Disponibilité de matière grasse en quantité (g/personne/jour),Disponibilité de protéines en quantité (g/personne/jour),Disponibilité intérieure,Exportations - Quantité,Importations - Quantité,Nourriture,Pertes,Production,Semences,Traitement,Variation de stock
0,Afghanistan,2,2511,Blé,vegetal,NaN,NaN,1369.0,160.23,4.69,36.91,5.992000e+09,NaN,1.173000e+09,4.895000e+09,775000000.0,5.169000e+09,322000000.0,NaN,-350000000.0


In [16]:
df_pop=pd.merge(df, population, on=["Zone", "Code zone"], how = 'inner')
df_pop

,Zone,Code zone,Code Produit,Produit,origine,Aliments pour animaux,Autres utilisations (non alimentaire),Disponibilité alimentaire (Kcal/personne/jour),Disponibilité alimentaire en quantité (kg/personne/an),Disponibilité de matière grasse en quantité (g/personne/jour),...,Exportations - Quantité,Importations - Quantité,Nourriture,Pertes,Production,Semences,Traitement,Variation de stock,Année,population
0,Afghanistan,2,2511,Blé,vegetal,NaN,NaN,1369.0,160.23,4.69,...,NaN,1.173000e+09,4.895000e+09,775000000.0,5.169000e+09,322000000.0,NaN,-350000000.0,2013,30552000
1,Afghanistan,2,2513,Orge,vegetal,360000000.0,NaN,26.0,2.92,0.24,...,NaN,1.000000e+07,8.900000e+07,52000000.0,5.140000e+08,22000000.0,NaN,0.0,2013,30552000
2,Afghanistan,2,2514,Maïs,vegetal,200000000.0,NaN,21.0,2.50,0.30,...,0.0,1.000000e+06,7.600000e+07,31000000.0,3.120000e+08,5000000.0,NaN,NaN,2013,30552000
3,Afghanistan,2,2517,Millet,vegetal,NaN,NaN,3.0,0.40,0.02,...,NaN,NaN,1.200000e+07,1000000.0,1.300000e+07,0.0,NaN,NaN,2013,30552000
4,Afghanistan,2,2520,"Céréales, Autres",vegetal,NaN,NaN,0.0,0.00,0.00,...,NaN,0.000000e+00,0.000000e+00,NaN,NaN,NaN,NaN,NaN,2013,30552000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15320,Îles Salomon,25,2781,Huiles de Poissons,animal,NaN,0.0,0.0,0.00,0.00,...,NaN,0.000000e+00,0.000000e+00,NaN,NaN,NaN,NaN,NaN,2013,561000
15321,Îles Salomon,25,2782,Huiles de Foie de Poisso,animal,NaN,0.0,0.0,0.00,0.00,...,NaN,0.000000e+00,0.000000e+00,NaN,NaN,NaN,NaN,NaN,2013,561000
15322,Îles Salomon,25,2805,Riz (Eq Blanchi),vegetal,0.0,12000000.0,623.0,63.76,1.36,...,0.0,4.700000e+07,3.600000e+07,1000000.0,3.000000e+06,0.0,NaN,0.0,2013,561000
15323,Îles Salomon,25,2848,Lait - Excl Beurre,animal,0.0,NaN,19.0,11.52,0.70,...,NaN,5.000000e+06,6.000000e+06,0.0,1.000000e+06,NaN,NaN,NaN,2013,561000


In [17]:
df_pop.shape

(15325, 22)

##### Création des colonnes Tot_dispo_alim_kg, Tot_dispo_alim_tonnes, Tot_dispo_prot_kg, Tot_dispo_alim_kcal

In [18]:
# Conversion disponibilité alimentaire kg/pers ---> kg
df_pop["Tot_dispo_alim_kg"] = df_pop["Disponibilité alimentaire en quantité (kg/personne/an)"] * df_pop["population"]

# Conversion disponibilité alimentaire kg ---> tonnes
df_pop["Tot_dispo_alim_tonnes"] = df_pop["Tot_dispo_alim_kg"] / 1000

# Conversion disponibilité en protéines g/pers/jour ---> kg
df_pop["Tot_dispo_prot_kg"] = df_pop["Disponibilité de protéines en quantité (g/personne/jour)"] / 1000 * df_pop["population"] * 365

# Conversion disponibilité alimentaire kcal/pers/j ---> kcal
df_pop["Tot_dispo_alim_kcal"] = df_pop["Disponibilité alimentaire (Kcal/personne/jour)"] * df_pop["population"] * 365


In [19]:
df_pop.shape

(15325, 26)

# Question 4

In [20]:
# Calcul du ratio énergie / masse
df_pop["ratio_kcal_kg"] = df_pop["Tot_dispo_alim_kcal"] / df_pop["Tot_dispo_alim_kg"]

# Calcul du ratio protéines / masse
df_pop["ratio_prot"] = df_pop["Tot_dispo_prot_kg"] / df_pop["Tot_dispo_alim_kg"]

# Remplacement des valeurs " infinies " par des NaN
df_pop = df_pop.replace([np.inf, -np.inf], np.nan)
df_pop

,Zone,Code zone,Code Produit,Produit,origine,Aliments pour animaux,Autres utilisations (non alimentaire),Disponibilité alimentaire (Kcal/personne/jour),Disponibilité alimentaire en quantité (kg/personne/an),Disponibilité de matière grasse en quantité (g/personne/jour),...,Traitement,Variation de stock,Année,population,Tot_dispo_alim_kg,Tot_dispo_alim_tonnes,Tot_dispo_prot_kg,Tot_dispo_alim_kcal,ratio_kcal_kg,ratio_prot
0,Afghanistan,2,2511,Blé,vegetal,NaN,NaN,1369.0,160.23,4.69,...,NaN,-350000000.0,2013,30552000,4.895347e+09,4895346.96,4.116011e+08,1.526638e+13,3118.548337,0.084080
1,Afghanistan,2,2513,Orge,vegetal,360000000.0,NaN,26.0,2.92,0.24,...,NaN,0.0,2013,30552000,8.921184e+07,89211.84,8.809669e+06,2.899385e+11,3250.000000,0.098750
2,Afghanistan,2,2514,Maïs,vegetal,200000000.0,NaN,21.0,2.50,0.30,...,NaN,NaN,2013,30552000,7.638000e+07,76380.00,6.244829e+06,2.341811e+11,3066.000000,0.081760
3,Afghanistan,2,2517,Millet,vegetal,NaN,NaN,3.0,0.40,0.02,...,NaN,NaN,2013,30552000,1.222080e+07,12220.80,8.921184e+05,3.345444e+10,2737.500000,0.073000
4,Afghanistan,2,2520,"Céréales, Autres",vegetal,NaN,NaN,0.0,0.00,0.00,...,NaN,NaN,2013,30552000,0.000000e+00,0.00,0.000000e+00,0.000000e+00,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15320,Îles Salomon,25,2781,Huiles de Poissons,animal,NaN,0.0,0.0,0.00,0.00,...,NaN,NaN,2013,561000,0.000000e+00,0.00,0.000000e+00,0.000000e+00,NaN,NaN
15321,Îles Salomon,25,2782,Huiles de Foie de Poisso,animal,NaN,0.0,0.0,0.00,0.00,...,NaN,NaN,2013,561000,0.000000e+00,0.00,0.000000e+00,0.000000e+00,NaN,NaN
15322,Îles Salomon,25,2805,Riz (Eq Blanchi),vegetal,0.0,12000000.0,623.0,63.76,1.36,...,NaN,0.0,2013,561000,3.576936e+07,35769.36,2.231938e+06,1.275686e+11,3566.420954,0.062398
15323,Îles Salomon,25,2848,Lait - Excl Beurre,animal,0.0,NaN,19.0,11.52,0.70,...,NaN,NaN,2013,561000,6.462720e+06,6462.72,2.150033e+05,3.890535e+09,601.996528,0.033268


##### Verification du ration énergie / masse des Oeufs

In [21]:
oeufs = df_pop[df_pop["Produit"] == "Oeufs"]
oeufs = oeufs[["Zone", "Code zone", "Produit", "ratio_kcal_kg"]]

# Nettoyage des NaN et des valeurs nulles
oeufs = oeufs.dropna()
oeufs = oeufs[oeufs["ratio_kcal_kg"] != 0]
print(" ")

print("Valeur nutritionnelle moyenne d'un oeuf (Wikipédia): 1470 kcal / kg")
print("Moyenne mondiale du ratio énergie / masse pour les 'Oeufs' : {:.0f}"\
      .format(oeufs["ratio_kcal_kg"].mean()), "kcal / kg")

 
Valeur nutritionnelle moyenne d'un oeuf (Wikipédia): 1470 kcal / kg
Moyenne mondiale du ratio énergie / masse pour les 'Oeufs' : 1364 kcal / kg


##### Vérification du ratio protéines / masse de l'Avoine

In [22]:
avoine = df_pop[df_pop["Produit"] == "Avoine"]
avoine = avoine[["Zone", "Code zone", "Produit", "ratio_prot"]]

# Nettoyage des NaN et des valeurs nulles
avoine = avoine.dropna()
avoine = avoine[avoine["ratio_prot"] != 0]
print("Nombre de valeur dans la table 'avoine' après nettoyage :", len(avoine.index))
print(" ")

print("Ratio protéines / poids de l'avoine (Wikipédia) : 10,7 %")
print("Moyenne mondiale du ratio protéines / poids pour l'Avoine : {:.1f}".format(avoine["ratio_prot"].mean() * 100), "%")

Nombre de valeur dans la table 'avoine' après nettoyage : 114
 
Ratio protéines / poids de l'avoine (Wikipédia) : 10,7 %
Moyenne mondiale du ratio protéines / poids pour l'Avoine : 8.8 %


# Question 5

### A/ Produits les plus caloriques - (ratio énergie / masse le plus élevé)

In [23]:
classement_ratio_kcal = df_pop[["Zone", "Code zone", "Année", "Produit", "origine", "ratio_kcal_kg"]]

# Nettoyage des NaN et des valeurs nulles
classement_ratio_kcal = classement_ratio_kcal.dropna()
classement_ratio_kcal = classement_ratio_kcal[classement_ratio_kcal["ratio_kcal_kg"] != 0]
print("Nombre de valeur dans la table 'classement_ratio_kcal' après nettoyage :", len(classement_ratio_kcal.index))
print(" ")

# Calcul de la moyenne par produit et classement des 20 produits les plus riches
classement_ratio_kcal = classement_ratio_kcal.groupby(["Produit", "origine"])["ratio_kcal_kg"].mean().sort_values(ascending=False).head(20)

# Affichage du classement
classement_ratio_kcal

Nombre de valeur dans la table 'classement_ratio_kcal' après nettoyage : 9672
 


Produit                   origine
Huiles de Foie de Poisso  animal     11809.003531
Huile de Sésame           vegetal    10192.188616
Huile de Son de Riz       vegetal     9314.765053
Huile d'Olive             vegetal     9301.823385
Huile de Germe de Maïs    vegetal     9278.454590
Huile de Colza&Moutarde   vegetal     9157.170010
Huile d'Arachide          vegetal     9118.384270
Huile de Tournesol        vegetal     9041.569959
Huile de Palmistes        vegetal     9010.065306
Huile Graines de Coton    vegetal     8967.670727
Huiles de Poissons        animal      8917.431319
Huile de Coco             vegetal     8913.579691
Huile de Soja             vegetal     8901.300904
Huile de Palme            vegetal     8791.492506
Huil Plantes Oleif Autr   vegetal     8714.007499
Graisses Animales Crue    animal      7590.532115
Beurre, Ghee              animal      7489.824503
Sésame                    vegetal     6142.036023
Arachides Decortiquees    vegetal     5705.037380
Palmistes       

### B/: 5 aliments les plus riches en protéines - (ratio protéines / masse le plus élevé)

In [24]:
classement_ratio_prot = df_pop[["Zone", "Code zone", "Année", "Produit", "origine", "ratio_prot"]]

# Nettoyage des NaN et des valeurs nulles
classement_ratio_prot = classement_ratio_prot.dropna()
classement_ratio_prot = classement_ratio_prot[classement_ratio_prot["ratio_prot"] != 0]
print("Nombre de valeur dans la table 'classement_ratio_prot' après nettoyage :", len(classement_ratio_prot.index))
print(" ")

# Calcul de la moyenne par produit et classement des 20 produits les plus riches
classement_ratio_prot = classement_ratio_prot.groupby(["Produit", "origine"])["ratio_prot"] \
                        .mean().sort_values(ascending=False).head(20)

# Affichage du classement
classement_ratio_prot

Nombre de valeur dans la table 'classement_ratio_prot' après nettoyage : 8606
 


Produit                   origine
Graines Colza/Moutarde    vegetal    0.365000
Soja                      vegetal    0.272457
Arachides Decortiquees    vegetal    0.250818
Pois                      vegetal    0.224493
Légumineuses Autres       vegetal    0.223418
Haricots                  vegetal    0.218899
Plantes Oleiferes, Autre  vegetal    0.187886
Sésame                    vegetal    0.187408
Viande, Autre             animal     0.178138
Abats Comestible          animal     0.176206
Aliments pour enfants     vegetal    0.154416
Cephalopodes              animal     0.150577
Viande de Bovins          animal     0.144569
Viande d'Ovins/Caprins    animal     0.139016
Viande de Volailles       animal     0.126236
Piments                   vegetal    0.124413
Poissons Pelagiques       animal     0.122987
Graines de tournesol      vegetal    0.120311
Poissons Marins, Autres   animal     0.113485
Poivre                    vegetal    0.112019
Name: ratio_prot, dtype: float64

# Question 6  - Disponibilité intérieure mondiale végétale

In [25]:
# Récupération des données
dispo_int_veg_kcal = df_pop[df_pop["origine"] == "vegetal"]

# Nettoyage des NaN et des valeurs nulles
dispo_int_veg_kcal = dispo_int_veg_kcal.dropna()
print("Disponibilité intérieure mondiale de produits d'origine végétale : {:.2E}"\
      .format(dispo_int_veg_kcal["Tot_dispo_alim_kcal"].sum()), "kcal")

Disponibilité intérieure mondiale de produits d'origine végétale : 1.95E+15 kcal


# Question 7 : Combien d'humains pourraient être nourris si toute la disponibilité intérieure mondiale de produits végétaux était utilisée pour de la nourriture ?

In [26]:
#dispo_int_veg_prot = df_pop[df_pop["origine"] == "vegetal"]
dispo_int_veg_prot = df_pop.loc[df_pop["origine"] == "vegetal", :].copy()

print("Disponibilité intérieure mondiale de produits d'origine végétale : {:.2E}"\
      .format(dispo_int_veg_prot["Tot_dispo_prot_kg"].sum()), "kg de protéines")

Disponibilité intérieure mondiale de produits d'origine végétale : 1.25E+11 kg de protéines


In [27]:
# Conversion du besoin en protéines /jour --> /an
besoin_prot_p = 0.060 * 365
nb_h = dispo_int_veg_prot["Tot_dispo_prot_kg"].sum() / besoin_prot_p

print("Disponibilité intérieure mondiale en protéines de produits d'origine végétale en nombre d'humains :{:.2E}"\
      .format(nb_h))
print("Pourcentage de la population mondiale : {:.1f}".format(nb_h / population["population"].sum() * 100), "%")

Disponibilité intérieure mondiale en protéines de produits d'origine végétale en nombre d'humains :5.72E+09
Pourcentage de la population mondiale : 81.7 %


# Question 8 : Combien d'humains pourraient être nourris si toute la disponibilité alimentaire en produits végétaux destinée aux animaux, et les pertes de produits végétaux étaient utilisés pour de la nourriture?

In [28]:
# Sur DF dispoVegeInt, ajouter une colonne de la somme Nourriture + Aliments pour animaux + Pertes
dispo_int_veg_prot['Sum_VegePertes'] = dispo_int_veg_prot[['Tot_dispo_alim_kg','Aliments pour animaux','Pertes']].sum(axis=1)

# Somme totale de Nourriture + Aliments pour animaux + Pertes
vege_kcal1 = dispo_int_veg_prot['Sum_VegePertes'].sum()

# Calcul du nombre d'humains qui pourraient être nourris en terme de KCAL, sous forme de % de la population mondiale
vege_kcal1 = (dispo_int_veg_prot['Sum_VegePertes'] * dispo_int_veg_prot['ratio_kcal_kg']).sum() / (2300*365)
print(" ")
print('La disponibilité alimentaire en produits végétaux en termes de calories, est de', round(vege_kcal1),\
      'kcal; ce qui pourrait nourrir', round(vege_kcal1*100/population['population'].sum(), 1), '% de la population mondiale.')
print(" ")

 
La disponibilité alimentaire en produits végétaux en termes de calories, est de 10922883198 kcal; ce qui pourrait nourrir 156.1 % de la population mondiale.
 


In [29]:
# Calculer la disponibilité intérieure en KGPROT dans le DF dispoVegeInt
dispo_int_veg_prot['Sum_Vege_Kgprot'] = dispo_int_veg_prot['Sum_VegePertes'] * dispo_int_veg_prot['ratio_prot']

# Calcul du nombre d'humains qui pourraient être nourris en terme de PROTEINES, sous forme de % de la population mondiale
vege_kgprot1 = dispo_int_veg_prot['Sum_Vege_Kgprot'].sum()/(0.06*365)*100
print(" ")
print('La disponibilité alimentaire en produits végétaux en termes de protéines, est de', round(vege_kgprot1),\
      'kg; ce qui pourrait nourrir', round(vege_kgprot1/population['population'].sum(), 1), '% de la population mondiale.')
print(" ")

 
La disponibilité alimentaire en produits végétaux en termes de protéines, est de 932744674318 kg; ce qui pourrait nourrir 133.3 % de la population mondiale.
 


# Question 9: Combien d'humains pourraient être nourris avec la disponibilité alimentaire mondiale

### En KCAL 

In [30]:
# Nombre d'humains (rappel : 2300 kcal / personne)
besoin_kcal_p = 2300*365
nb_h = df_pop["Tot_dispo_alim_kcal"].sum() / besoin_kcal_p

print("{:.2E}".format(df_pop["Tot_dispo_alim_kcal"].sum()))
print("Disponibilité alimentaire mondiale en énergie (tous produits confondus) en nombre d'humains : {:.2E}".format(nb_h))
print("Pourcentage de la population mondiale : {:.1f}".format(nb_h / population["population"].sum() * 100), "%")

7.36E+15
Disponibilité alimentaire mondiale en énergie (tous produits confondus) en nombre d'humains : 8.77E+09
Pourcentage de la population mondiale : 125.4 %


### En Proteines 

In [31]:
# Nombre d'humains (rappel : 0,060 kg / personne)
nb_h = df_pop["Tot_dispo_prot_kg"].sum() / besoin_prot_p

print("Disponibilité alimentaire mondiale en énergie (tous produits confondus) en nombre d'humains : {:.2E}".format(nb_h))
print("Pourcentage de la population mondiale : {:.1f}".format(nb_h / population["population"].sum() * 100), "%")

Disponibilité alimentaire mondiale en énergie (tous produits confondus) en nombre d'humains : 9.46E+09
Pourcentage de la population mondiale : 135.2 %


# Question 10 - Proportion de sous alimentés

In [32]:
proportionSousAlimentes = sousalimentation[sousalimentation['Année'] == 2013]['pop_sousalim'].sum()
print(int(proportionSousAlimentes), 'de personnes sous-nutries en 2013')

743800000 de personnes sous-nutries en 2013


In [33]:
# On calcul le ratio de personnes sous-alimentées sur la population totale
ratioPopulationSousAlimentes = ((proportionSousAlimentes) / totalPopulation)*100

print('Proportion de la population sous-alimentées en 2013', round(ratioPopulationSousAlimentes,2), "%")

Proportion de la population sous-alimentées en 2013 10.63 %


# Question 11 - Céréales alimentation animale

In [34]:
cereales = pd.read_csv('fr_cereales.csv')

### Etablir la liste des produits + codes produits, considéré comme des céréales selon FAO. Pour cela on créée le DF cerealesListe

In [35]:
cerealesListe = pd.DataFrame({'Céréales Produit': cereales['Produit'].unique(),'Céréales Code': cereales['Code Produit'].unique()})
cerealesListe

,Céréales Produit,Céréales Code
0,Blé,2511
1,Riz (Eq Blanchi),2805
2,Orge,2513
3,Maïs,2514
4,Millet,2517
5,Seigle,2515
6,Avoine,2516
7,Sorgho,2518
8,"Céréales, Autres",2520


### On créée une colonne sur df_pop 'is_cereal' BOOLEAN

In [36]:
df_pop['is_cereal'] = df_pop['Produit'].isin(cerealesListe['Céréales Produit'])

In [37]:
df_pop.head(3)

,Zone,Code zone,Code Produit,Produit,origine,Aliments pour animaux,Autres utilisations (non alimentaire),Disponibilité alimentaire (Kcal/personne/jour),Disponibilité alimentaire en quantité (kg/personne/an),Disponibilité de matière grasse en quantité (g/personne/jour),...,Variation de stock,Année,population,Tot_dispo_alim_kg,Tot_dispo_alim_tonnes,Tot_dispo_prot_kg,Tot_dispo_alim_kcal,ratio_kcal_kg,ratio_prot,is_cereal
0,Afghanistan,2,2511,Blé,vegetal,NaN,NaN,1369.0,160.23,4.69,...,-350000000.0,2013,30552000,4.895347e+09,4895346.96,411601126.8,1.526638e+13,3118.548337,0.08408,True
1,Afghanistan,2,2513,Orge,vegetal,360000000.0,NaN,26.0,2.92,0.24,...,0.0,2013,30552000,8.921184e+07,89211.84,8809669.2,2.899385e+11,3250.000000,0.09875,True
2,Afghanistan,2,2514,Maïs,vegetal,200000000.0,NaN,21.0,2.50,0.30,...,NaN,2013,30552000,7.638000e+07,76380.00,6244828.8,2.341811e+11,3066.000000,0.08176,True


In [38]:
# Calcul du poids des aliments humain et nourriture animale 
pourcentageNourritureAnimale = df_pop[df_pop['is_cereal'] == True]['Aliments pour animaux'].sum() 
pourcentageNourritureHumaine = df_pop[df_pop['is_cereal'] == True]['Nourriture'].sum()

# Calcul du taux de nourriture animale
tauxNourritureAnimale = (pourcentageNourritureAnimale / (pourcentageNourritureAnimale + pourcentageNourritureHumaine))*100
print('La part de nourriture animale sur la part globale de céréales à destination humaine et animale est de',
      round(tauxNourritureAnimale,2), '%')

La part de nourriture animale sur la part globale de céréales à destination humaine et animale est de 45.91 %


# Question 12 : Donnez les 3 produits qui ont la plus grande valeur pour chacun des 2 ratios

In [39]:
#Création de la liste de tous les pays en sous alimentation
liste_pays = sousalimentation.loc[sousalimentation['pop_sousalim'] > 0, "Zone"].unique()

In [40]:
#Création du DF temporary, à partir de notre df_pop, qui contient tous les pays en sous alimentation
temporary = df_pop.loc[df_pop['Zone'].isin(liste_pays), :]

In [41]:
# Groupby par produit
temporary = temporary.groupby('Produit').sum().reset_index()

In [42]:
# Afficher les 20 premieres
temporary = temporary.sort_values('Exportations - Quantité', ascending=False)[:20]
display(temporary)

,Produit,Code zone,Code Produit,Aliments pour animaux,Autres utilisations (non alimentaire),Disponibilité alimentaire (Kcal/personne/jour),Disponibilité alimentaire en quantité (kg/personne/an),Disponibilité de matière grasse en quantité (g/personne/jour),Disponibilité de protéines en quantité (g/personne/jour),Disponibilité intérieure,...,Variation de stock,Année,population,Tot_dispo_alim_kg,Tot_dispo_alim_tonnes,Tot_dispo_prot_kg,Tot_dispo_alim_kcal,ratio_kcal_kg,ratio_prot,is_cereal
52,Maïs,12595,243858,2.744160e+11,4.611700e+10,24909.0,2932.88,245.74,636.35,4.745890e+11,...,-1.350800e+10,195261,5423772000,1.060028e+11,1.060028e+08,8.061106e+09,3.222986e+14,282378.910618,7.285193,97
39,Huile de Palme,12353,244815,0.000000e+00,2.500700e+10,6790.0,282.01,767.82,0.29,3.811100e+10,...,-8.750000e+08,191235,5414928000,1.294524e+10,1.294524e+07,7.982941e+06,1.128211e+14,635531.605602,0.019863,0
51,Manioc,11682,230412,7.573700e+10,3.409300e+10,8523.0,3082.11,11.48,63.65,2.311490e+11,...,-6.560000e+08,183183,5229106000,9.293481e+10,9.293481e+07,5.939556e+08,8.942247e+13,60637.632840,0.441091,0
12,Blé,12595,243567,5.232700e+10,1.015100e+10,45612.0,5776.83,222.13,1327.76,4.131920e+11,...,-6.741000e+09,195261,5423772000,3.150969e+11,3.150969e+08,2.831772e+10,9.500951e+14,277780.655177,7.956274,97
79,Riz (Eq Blanchi),12595,272085,3.292700e+10,1.785900e+10,38179.0,3908.16,92.11,747.96,4.437740e+11,...,-1.673000e+10,195261,5423772000,3.505168e+11,3.505168e+08,2.399140e+10,1.279286e+15,350405.569829,6.785352,97
83,Sucre Eq Brut,12595,246574,2.300000e+07,6.101000e+09,20694.0,2120.87,0.00,0.00,1.091980e+11,...,-3.576000e+09,195261,5423772000,9.141472e+10,9.141472e+07,0.000000e+00,3.242536e+14,347437.353124,0.000000,0
49,"Légumes, Autres",12595,252685,3.965200e+10,7.800000e+07,3618.0,5226.10,29.30,178.54,7.404790e+11,...,1.550000e+08,195261,5423772000,6.449813e+11,6.449813e+08,8.692867e+09,1.580428e+14,25790.018220,1.288043,0
8,Bananes,11882,240580,8.280000e+08,1.180000e+08,2164.0,1320.59,7.61,29.75,8.355700e+10,...,4.800000e+08,185196,5160671000,6.756640e+10,6.756640e+07,5.606898e+08,4.200823e+13,50323.367080,0.738937,0
81,Soja,11935,229950,1.255300e+10,6.380000e+08,553.0,56.95,24.79,54.76,1.501640e+11,...,-5.135000e+09,181170,5365888000,8.184880e+09,8.184880e+06,2.709877e+09,2.867285e+13,126317.421223,17.638345,0
48,Lait - Excl Beurre,12595,276256,4.323200e+10,2.678000e+09,11399.0,6908.42,620.87,641.31,3.808650e+11,...,2.600000e+08,195261,5423772000,3.207688e+11,3.207688e+08,1.108435e+10,1.886081e+14,59012.700312,3.268387,0


In [43]:
# Création DF liste_produits
liste_produit = temporary['Produit']
liste_produit

52                   Maïs
39         Huile de Palme
51                 Manioc
12                    Blé
79       Riz (Eq Blanchi)
83          Sucre Eq Brut
49        Légumes, Autres
8                 Bananes
81                   Soja
48     Lait - Excl Beurre
25         Fruits, Autres
73    Poissons Pelagiques
89                Tomates
62                   Orge
75                 Pommes
61    Oranges, Mandarines
4                  Ananas
41          Huile de Soja
15                   Café
57                   Noix
Name: Produit, dtype: object

In [44]:
# Créer DF temporary (temporaire) pour les produits 
temporary = df_pop.loc[df_pop['Produit'].isin(liste_produit), :]

In [45]:
# Afficher les 200 premiers
temporary = temporary.sort_values('Importations - Quantité', ascending=False)[:200]
display(temporary)

,Zone,Code zone,Code Produit,Produit,origine,Aliments pour animaux,Autres utilisations (non alimentaire),Disponibilité alimentaire (Kcal/personne/jour),Disponibilité alimentaire en quantité (kg/personne/an),Disponibilité de matière grasse en quantité (g/personne/jour),...,Variation de stock,Année,population,Tot_dispo_alim_kg,Tot_dispo_alim_tonnes,Tot_dispo_prot_kg,Tot_dispo_alim_kcal,ratio_kcal_kg,ratio_prot,is_cereal
3014,Chine,351,2555,Soja,vegetal,9.530000e+09,1.000000e+06,36.0,3.87,1.24,...,326000000.0,2013,1416667000,5.482501e+09,5482501.29,1.763255e+09,1.861500e+13,3395.348837,0.321615,False
3001,Chine,351,2532,Manioc,vegetal,2.287300e+10,9.257000e+09,6.0,1.88,0.01,...,63000000.0,2013,1416667000,2.663334e+09,2663333.96,2.585417e+07,3.102501e+12,1164.893617,0.009707,False
6744,Japon,110,2514,Maïs,vegetal,1.096400e+10,2.880000e+08,61.0,9.21,0.20,...,259000000.0,2013,127144000,1.170996e+09,1170996.24,7.425210e+06,2.830861e+12,2417.480999,0.006341,True
607,Arabie saoudite,194,2513,Orge,vegetal,1.000000e+10,NaN,0.0,0.01,0.00,...,-241000000.0,2013,28829000,2.882900e+05,288.29,0.000000e+00,0.000000e+00,0.000000,0.000000,True
14779,Égypte,59,2511,Blé,vegetal,4.860000e+09,0.000000e+00,1175.0,146.83,6.27,...,-260000000.0,2013,82056000,1.204828e+10,12048282.48,1.066835e+09,3.519177e+13,2920.894912,0.088547,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11220,Royaume-Uni,229,2615,Bananes,vegetal,NaN,NaN,28.0,16.58,0.05,...,NaN,2013,63384000,1.050907e+09,1050906.72,8.328658e+06,6.477845e+11,616.405308,0.007925,False
4602,France,68,2601,Tomates,vegetal,NaN,NaN,10.0,20.45,0.11,...,NaN,2013,64291000,1.314751e+09,1314750.95,1.173311e+07,2.346622e+11,178.484108,0.008924,False
1071,Autriche,11,2848,Lait - Excl Beurre,animal,3.050000e+08,6.300000e+07,269.0,258.09,15.48,...,-55000000.0,2013,8495000,2.192475e+09,2192474.55,6.892801e+07,8.340816e+11,380.429308,0.031438,False
13243,Sénégal,195,2805,Riz (Eq Blanchi),vegetal,NaN,6.300000e+07,715.0,72.29,1.36,...,-214000000.0,2013,14133000,1.021675e+09,1021674.57,7.433463e+07,3.688360e+12,3610.112049,0.072758,True


In [46]:
# Réduire les colonnes en ne gardant que celles qui nous intéressent pour les calculs
temporary = temporary[['Produit','Tot_dispo_alim_kg','Aliments pour animaux','Autres utilisations (non alimentaire)','Disponibilité intérieure']]

In [47]:
print(temporary.columns.tolist())

['Produit', 'Tot_dispo_alim_kg', 'Aliments pour animaux', 'Autres utilisations (non alimentaire)', 'Disponibilité intérieure']


In [48]:
# Création DF produit_group
produit_group = temporary.groupby('Produit').sum()

In [49]:
# Calcul des ratios
produit_group['ratio_Autres_Utilisation'] = produit_group['Autres utilisations (non alimentaire)']/produit_group['Disponibilité intérieure']
produit_group['ratio_Alim_Animale'] = produit_group['Aliments pour animaux']/(produit_group['Aliments pour animaux']+produit_group['Tot_dispo_alim_kg'])

### 5 plus grandes valeurs pour le ratio entre la quantité destinés aux "Autres utilisations" (Other uses) et la disponibilité intérieure.


In [50]:
produit_group.sort_values('ratio_Autres_Utilisation', ascending=False).head(5)

,Tot_dispo_alim_kg,Aliments pour animaux,Autres utilisations (non alimentaire),Disponibilité intérieure,ratio_Autres_Utilisation,ratio_Alim_Animale
Produit,,,,,,
Huile de Soja,2.875834e+09,0.000000e+00,9.216000e+09,1.208900e+10,0.762346,0.000000
Huile de Palme,6.924490e+09,0.000000e+00,1.872600e+10,2.586500e+10,0.723990,0.000000
Manioc,3.539896e+09,2.467300e+10,1.146100e+10,4.312200e+10,0.265781,0.874529
Maïs,5.528330e+10,3.921640e+11,1.818690e+11,6.878210e+11,0.264413,0.876447
Sucre Eq Brut,6.581438e+10,0.000000e+00,3.009000e+09,7.932100e+10,0.037934,0.000000


### 5 plus grandes valeurs pour le ratio entre la quantité destinée à la nourriture animale et la quantité destinée à la nourriture (animale + humaine)


In [51]:
produit_group.sort_values('ratio_Alim_Animale', ascending=False).head(5)

,Tot_dispo_alim_kg,Aliments pour animaux,Autres utilisations (non alimentaire),Disponibilité intérieure,ratio_Autres_Utilisation,ratio_Alim_Animale
Produit,,,,,,
Orge,3.719872e+08,2.016000e+10,1.200000e+07,2.960000e+10,0.000405,0.981883
Maïs,5.528330e+10,3.921640e+11,1.818690e+11,6.878210e+11,0.264413,0.876447
Manioc,3.539896e+09,2.467300e+10,1.146100e+10,4.312200e+10,0.265781,0.874529
Poissons Pelagiques,4.057592e+09,1.309700e+10,4.000000e+07,1.719000e+10,0.002327,0.763469
Soja,7.529069e+09,1.371000e+10,5.250000e+08,1.595930e+11,0.003290,0.645509


# Question 13: Combien de tonnes de céréales pourraient être libérées si les USA diminuaient leur production de produits animaux de 10% ?

In [52]:
dispoUSA = df_pop.loc[(df_pop['Zone'] == "États-Unis d'Amérique") & (df_pop['is_cereal']),:]
print('Si les USA diminuaient de 10% leur production de produits animaux, cela libèrerait :', int(dispoUSA['Aliments pour animaux'].sum()*0.1/1000), 'Tonnes de céréales')

Si les USA diminuaient de 10% leur production de produits animaux, cela libèrerait : 14009600 Tonnes de céréales


# Question 14: En Thaïlande, quelle proportion de manioc est exportée ? Quelle est la proportion de personnes en sous-nutrition?

### Proportion de manioc exportée

In [53]:
proportionManiocExporte = df_pop.loc[(df_pop['Zone'] == "Thaïlande") & (df_pop['Produit'] == 'Manioc'),:]
print('La Thaïlande exporte', round(proportionManiocExporte['Exportations - Quantité'].iloc[0]*100/proportionManiocExporte['Production'].iloc[0], 1), "% du Manioc qu'elle produit.")


La Thaïlande exporte 83.4 % du Manioc qu'elle produit.


### Proportion de personnes en sous-nutrition

In [54]:
df_pop.columns.tolist()

['Zone',
 'Code zone',
 'Code Produit',
 'Produit',
 'origine',
 'Aliments pour animaux',
 'Autres utilisations (non alimentaire)',
 'Disponibilité alimentaire (Kcal/personne/jour)',
 'Disponibilité alimentaire en quantité (kg/personne/an)',
 'Disponibilité de matière grasse en quantité (g/personne/jour)',
 'Disponibilité de protéines en quantité (g/personne/jour)',
 'Disponibilité intérieure',
 'Exportations - Quantité',
 'Importations - Quantité',
 'Nourriture',
 'Pertes',
 'Production',
 'Semences',
 'Traitement',
 'Variation de stock',
 'Année',
 'population',
 'Tot_dispo_alim_kg',
 'Tot_dispo_alim_tonnes',
 'Tot_dispo_prot_kg',
 'Tot_dispo_alim_kcal',
 'ratio_kcal_kg',
 'ratio_prot',
 'is_cereal']

In [55]:
#On prends la population de la Thailande. iloc car dans une ligne on a directement la valeur de la population
populationThailande = df_pop.loc[df_pop['Zone']=='Thaïlande','population'].iloc[0]

#On prends la population sousalimentée de la Thailande.
sousAlimentationThailande = sousalimentation.loc[sousalimentation['Zone']=='Thaïlande','pop_sousalim'].iloc[0]

#On fait un ratio des deux.
print(round(sousAlimentationThailande*100/populationThailande,1), '% de la population de la Thaïlande est considérée comme sous-alimentée.')

8.4 % de la population de la Thaïlande est considérée comme sous-alimentée.


# B/ PARTIE SQL

# Question 15 : Proposez une clé primaire pertinente pour la table population

In [56]:
population = population.rename(columns={"Zone": "pays", "Code zone": "code_pays", "Année": "annee"})
population.to_csv('Population_nettoyee.csv')

# Question 16 : Proposez une clé primaire pertinente pour la table dispo_alim

In [57]:
dispo_alim = df_pop.rename(columns = {"Zone": "pays",\
                                      "Code zone": "code_pays",\
                                      "Année": "annee",\
                                      "Produit":"produit",\
                                      "Disponibilité alimentaire (Kcal/personne/jour)":"dispo_alim_kcal_p_j",\
                                      'Disponibilité de protéines en quantité (g/personne/jour)':'dispo_prot',\
                                      'Disponibilité de matière grasse en quantité (g/personne/jour)':'dispo_mat_gr',\
                                      'Tot_dispo_alim_tonnes':'dispo_alim_tonnes',\
                                      'Code Produit':'code_produit'})

In [58]:
dispo_alim.drop(columns=['Aliments pour animaux',\
                         'Autres utilisations (non alimentaire)',\
                         'Exportations - Quantité',\
                         'Importations - Quantité',\
                         'Nourriture',\
                         'Pertes',\
                         'Production',\
                         'Semences',\
                         'Traitement',\
                         'Variation de stock',\
                         'population',\
                         'ratio_kcal_kg',\
                         'ratio_prot',\
                         'is_cereal',
                         'Disponibilité alimentaire en quantité (kg/personne/an)',\
                         'Disponibilité intérieure',
                         'Tot_dispo_alim_kg',
                         'Tot_dispo_prot_kg',
                         'Tot_dispo_alim_kcal'
                         ],inplace=True)

In [59]:
dispo_alim.to_csv('dispo_alim.csv')

# Question 17 Proposez une clé primaire pertinente pour la table equilibre_prod

In [60]:
equilibre_prod = df_pop.rename(columns = {"Zone": "pays",\
                                          "Code zone": "code_pays",\
                                          "Année": "annee",\
                                          "Produit":"produit",\
                                          'Code Produit':'code_produit',\
                                          "Aliments pour animaux":"alim_ani",\
                                          "Semences":"semences",\
                                          'Nourriture':'nourriture',\
                                          'Pertes':'pertes',\
                                          "Disponibilité intérieure":'dispo_int',\
                                          'Autres utilisations (non alimentaire)':'autres_utilisations',\
                                          'Traitement':'transfo'                                         
                                         })

In [61]:
equilibre_prod.drop(columns=['Disponibilité alimentaire (Kcal/personne/jour)',\
                         'Disponibilité alimentaire en quantité (kg/personne/an)',\
                         'Disponibilité de matière grasse en quantité (g/personne/jour)',\
                         'Disponibilité de protéines en quantité (g/personne/jour)',\
                         'Variation de stock',\
                         'Production',\
                         'Variation de stock',\
                         'population',\
                         'ratio_kcal_kg',\
                         'ratio_prot',\
                         'is_cereal',
                         'Disponibilité alimentaire en quantité (kg/personne/an)',\
                         'Tot_dispo_alim_kg',
                         'Tot_dispo_prot_kg',
                         'Tot_dispo_alim_kcal',
                         'Exportations - Quantité',\
                         'Tot_dispo_alim_tonnes',\
                         'origine',\
                         'Importations - Quantité'
                         ],inplace=True)

In [62]:
equilibre_prod.to_csv('equilibre_prod.csv')

# Question 18 : Proposez une clé primaire pertinente pour la table sous_nutrition

In [63]:
sous_nutrition = sousalimentation.rename(columns={'Zone': 'pays', 'Code zone':'code_pays', 'Année':'annee', 'pop_sousalim':'nb_personnes'})
sous_nutrition= sous_nutrition[sous_nutrition['annee'] == 2013]
sous_nutrition

,pays,code_pays,annee,nb_personnes
0,Afghanistan,2,2013,7900000.0
5,Afrique du Sud,202,2013,2600000.0
10,Albanie,3,2013,200000.0
15,Algérie,4,2013,1700000.0
20,Allemagne,79,2013,NaN
...,...,...,...,...
995,Venezuela (République bolivarienne du),236,2013,1900000.0
1000,Viet Nam,237,2013,10400000.0
1005,Yémen,249,2013,7200000.0
1010,Zambie,251,2013,7000000.0


In [64]:
sous_nutrition.to_csv('sous_nutrition.csv')

# Question 19 : 

### Ecrire des requêtes SQL sur cette base de données relationnelle.

### Les 10 pays ayant le plus haut ratio disponibilité alimentaire/habitant en termes de protéines (en kg) par habitant, puis en termes de kcal par habitant.

### Les 10 pays ayant le plus faible ratio disponibilité alimentaire/habitant en termes de protéines (en kg) par habitant, puis en termes de kcal par habitant.

### La quantité totale (en kg) de produits perdus par pays en 2013.

### Les 10 pays pour lesquels la proportion de personnes sous-alimentées est la plus forte.

### Les 10 produits pour lesquels le ratio Autres utilisations/Disponibilité intérieure est le plus élevé.

# Question 20 : pour quelques uns des produits identifiés dans cette dernière requête SQL, supposez quelles sont ces "autres utilisations" possibles